In [1]:
import pandas as pd
import numpy as np
import pickle as p

In [2]:
data_all = pd.read_csv('data_all.csv')

In [3]:
np.random.seed(42)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

In [4]:
item_subset

[3722, 3718, 2721, 2256, 7009, 2257, 3732, 3727, 7115, 7157]

### `/train`

In [17]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [16]:
# Model Training Result
r.json()['result']

'Success'

### `/train`

In [27]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [28]:
r.json()['result']

[8.806467357931746,
 12.173855696137993,
 12.372778794421876,
 7.656308067942417,
 10.056290455246648,
 9.794202688557373,
 11.222999715105843,
 4.56828447948706,
 4.569695450613792,
 7.121983363800724]

In [9]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict(json.loads(r.json()['cv_acc']))
cv_accuracies

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,2256.0,57.811802,0.632769,13.758904,0.237995,18.183257
1,2257.0,18.398319,0.641186,5.163987,0.280677,6.866726
2,2721.0,26.516331,0.696132,9.607348,0.362318,14.734598
3,3718.0,252.716995,0.763973,40.389042,0.159819,53.661878
4,3722.0,367.495941,0.756677,56.299565,0.153198,75.614301
5,3727.0,296.811249,0.731813,49.405335,0.166454,65.059568
6,3732.0,127.388702,0.661943,24.491288,0.192256,32.776638
7,7009.0,2009.090332,0.781729,357.424910,0.177904,495.326576
8,7115.0,29.458530,0.526566,10.037080,0.340719,13.920075
9,7157.0,12.570630,0.429629,5.119320,0.407245,6.955913


### `/predict/`

In [15]:
# Craft mock data input, average historical prices of items
prices_input = {}
for item in item_subset:
    avg_item_price = data_subset.query('Item_ID=={}'.format(item))['Price_'].mean()
    prices_input[int(item)] = round(avg_item_price,2)
    
prices_input

{3722: 10.72,
 3718: 7.6,
 2721: 13.07,
 2256: 8.27,
 7009: 4.62,
 2257: 11.54,
 3732: 11.79,
 3727: 10.81,
 7115: 4.81,
 7157: 6.66}

In [16]:
import requests
import json

url = 'http://localhost:5000/predict/'



payload = {'prices': prices_input,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [17]:
# What If Prediction
r.json()['pred_q']

{'2256': 57,
 '2257': 18,
 '2721': 26,
 '3718': 291,
 '3722': 361,
 '3727': 288,
 '3732': 150,
 '7009': 1922,
 '7115': 34,
 '7157': 12}